In [ ]:
import pandas as pd

df = pd.read_csv('2016-2023-imi-results-long-view.csv')

new_df = df.dropna()
new_df.dropna(inplace = True)

df_filtered_cols = new_df.drop(['HOSPITAL', 'OSHPDID', 'Procedure/Condition',
                               'Risk Adjuested Mortality Rate',
                               'Hospital Ratings', 'LONGITUDE', 'LATITUDE'], axis=1)

df_filtered_cols["# of Deaths"] = pd.to_numeric(df_filtered_cols["# of Deaths"])
df_filtered_cols["# of Cases"] = pd.to_numeric(df_filtered_cols["# of Cases"])

df1 = df_filtered_cols.groupby(["YEAR","COUNTY"])["# of Deaths"].sum()
df2 = df_filtered_cols.groupby(["YEAR","COUNTY"])["# of Cases"].sum()

df_cleaned = pd.concat([df1, df2], axis=1)

print(df_cleaned)

df1.to_csv("cleaned_2016-2023-imi-results-long-view.csv")

                # of Deaths  # of Cases
YEAR COUNTY                            
2016 Alameda           1622       34012
     Amador              38         790
     Butte              394       10032
     Calaveras           10         310
     Colusa               2          14
...                     ...         ...
2023 Tulare             248        5089
     Tuolumne            23         671
     Ventura            413        9575
     Yolo                26         912
     Yuba                95        2265

[438 rows x 2 columns]


/tmp/ipython-input-157-1546423205.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace = True)


In [ ]:
cols_to_use = [0,2,3,4,6,8]
df = pd.read_csv('20230413_adult_demo_race_supp.csv', usecols= cols_to_use)

new_df = df.dropna()
new_df.dropna(inplace = True)

print(new_df)

new_df.to_csv("cleaned_20230413_adult_demo_race_supp.csv")

                POPULATION_NAME  FISCAL_YEAR  \
0                       Alameda         2019   
1                       Alameda         2019   
2                       Alameda         2019   
3                       Alameda         2019   
4                       Alameda         2019   
...                         ...          ...   
18361  Very Large Counties (LA)         2022   
18362  Very Large Counties (LA)         2022   
18363  Very Large Counties (LA)         2022   
18364  Very Large Counties (LA)         2022   
18365  Very Large Counties (LA)         2022   

                                   RACE_GRP   TOTAL_CT  MHS1_CT  MHS5_CT  
0      R1 Alaskan Native or American Indian      851.0    109.0     25.0  
1              R2 Asian or Pacific Islander    81208.0   4318.0    911.0  
2                                  R3 Black    53823.0   6484.0   1528.0  
3                               R4 Hispanic    73645.0   4357.0    891.0  
4                                  R5 White    4

/tmp/ipython-input-158-2163273563.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace = True)


In [ ]:
df = pd.read_csv('4.7-cohs-plans-q4-2024-odp.csv')

d = {}

d["CalOPTIMA/Orange"] = "Orange"
d["CalOptima/Orange"] = "Orange"
d["Central California Alliance for Health"] = "San Benito, Santa Cruz, Monterey, Merced, Mariposa"
d["Gold Coast Health Plan"] = "Ventura"
d["Partnership Health Plan of CA"] = "Del Norte, Colusa, Butte, Placer, Plumas, Siskiyou, Lassen, Solano, Lake, Sierra, Shasta, Glenn, Humboldt, Napa, Yolo, Nevada, Yuba, Trinity, Modoc, Sutter, Mendocino, Tehama, Sonoma, Marin"
d["Partnership Health Plan of CA/"] = "Del Norte, Colusa, Butte, Placer, Plumas, Siskiyou, Lassen, Solano, Lake, Sierra, Shasta, Glenn, Humboldt, Napa, Yolo, Nevada, Yuba, Trinity, Modoc, Sutter, Mendocino, Tehama, Sonoma, Marin"
d["Partnership Health Plan of California/"] = "Del Norte, Colusa, Butte, Placer, Plumas, Siskiyou, Lassen, Solano, Lake, Sierra, Shasta, Glenn, Humboldt, Napa, Yolo, Nevada, Yuba, Trinity, Modoc, Sutter, Mendocino, Tehama, Sonoma, Marin"
d["Santa Barbara Health Authority"] = "Santa Barbara"
d["CenCal - San Luis Obispo"] = "San Luis Obispo"
d["CenCal - Santa Barbara"] = "Santa Barbara"
d["CenCal Health/San Luis Obispo"] = "San Luis Obispo"
d["CenCal Health/Santa Barbara"] = "Santa Barbara"
d["Gold Coast Health Plan/Ventura"] = "Ventura"
d["Health Plan of San Mateo"] = "San Mateo"
d["Health Plan of San Mateo/San Mateo"] = "San Mateo"
d["Partnership Health Plan/Yuba"] = "Yuba"

for key, value in d.items():
  df["County Organized Health System (COHS) Health Plan"] = df["County Organized Health System (COHS) Health Plan"].apply(lambda x: value if x == key else x)

for company in df["County Organized Health System (COHS) Health Plan"]:
  if "Kaiser Permanente/" in company:
    county = company.split("/")[1]
  else:
    county = company
  df["County Organized Health System (COHS) Health Plan"] = df["County Organized Health System (COHS) Health Plan"].apply(lambda x: county if x == company else x)

for index, row in df.iterrows():
  year = row["Year"]
  period = row["Reporting Period"]
  county = row["County Organized Health System (COHS) Health Plan"]
  eligible = row[" Number of Eligible Individuals "]

  counties = county.split(", ")
  if len(counties) > 1:
    i = df[((df["Year"] == year)
    & (df["Reporting Period"] == period)
    & (df["County Organized Health System (COHS) Health Plan"] == county)
    & (df[" Number of Eligible Individuals "] == eligible))].index
    df.drop(i, inplace=True)

    for countyIndividual in counties:
      df.loc[len(df)] = [year, period, countyIndividual, eligible]



df_filtered_cols = df.drop(['Reporting Period'], axis=1)

df_filtered_cols[' Number of Eligible Individuals '] = df_filtered_cols[' Number of Eligible Individuals '].str.findall(r'[^\s,](?:[^,]*[^\s,])?').str.join('')
df_filtered_cols[" Number of Eligible Individuals "] = pd.to_numeric(df_filtered_cols[" Number of Eligible Individuals "])

df_filtered_cols = df_filtered_cols.groupby(["Year","County Organized Health System (COHS) Health Plan"])[" Number of Eligible Individuals "].sum()

new_df = df_filtered_cols.dropna()
new_df.dropna(inplace = True)

print(new_df)

new_df.to_csv("cleaned_4.7-cohs-plans-q4-2024-odp.csv")

Year  County Organized Health System (COHS) Health Plan
2016  Marin                                                11584
      Mariposa                                              6175
      Orange                                               21691
      San Mateo                                             2881
      Santa Barbara                                         3682
                                                           ...  
2024  Placer                                                  78
      San Luis Obispo                                        654
      San Mateo                                             1503
      Santa Barbara                                         1087
      Ventura                                               1535
Name:  Number of Eligible Individuals , Length: 61, dtype: int64


In [ ]:
df = pd.read_csv('data-infant-mortality.csv')

df_filtered_cols = df.drop(['Category', 'Subcategory',
                            'Rate', 'Indicator Name',
                               'Lower 95% Confidence Limit',
                               'Upper 95% Confidence Limit',
                                'Unnamed: 10', 'Unnamed: 11',
                                'Unnamed: 12', 'Unnamed: 13',
                                'Unnamed: 14'], axis=1)

df_filtered_cols = df_filtered_cols.iloc[2:].reset_index()
df_filtered_cols = df_filtered_cols.drop(['index'], axis=1)


df1 = df_filtered_cols.groupby(["Geography","Year"])["Numerator"].sum()
df2 = df_filtered_cols.groupby(["Geography","Year"])["Denominator"].sum()

df_cleaned = pd.concat([df1, df2], axis=1)

new_df = df_cleaned.dropna()
new_df.dropna(inplace = True)

print(new_df)
new_df.to_csv("cleaned_data-infant-mortality.csv")

                     Numerator  Denominator
Geography Year                             
Alameda   2007           107.0      21522.0
          2007-2009     4913.0    1378297.0
          2008            81.0      20977.0
          2008-2010     4804.0    1331159.0
          2009            82.0      20326.0
...                        ...          ...
Yuba      2018-2020      281.0      80819.0
          2019            12.0       1167.0
          2019-2021      278.0      82619.0
          2020             0.0       1116.0
          2021             0.0       1181.0

[1678 rows x 2 columns]
